In [1]:
from transformers import FastImageProcessor, FastForSceneTextRecognition, AutoConfig, AutoBackbone
from transformers import TextNetConfig
from transformers import TextNetBackbone
from PIL import Image
import requests

url = "https://huggingface.co/datasets/Raghavan/fast_model_samples/resolve/main/img657.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
processor = FastImageProcessor.from_pretrained("Raghavan/ic17mlt_Fast_T")


/home/user/miniconda/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
preprocessor_config.json: 100%|██████████| 486/486 [00:00<00:00, 3.80MB/s]


In [3]:
model = FastForSceneTextRecognition.from_pretrained("Raghavan/ic17mlt_Fast_T")

config.json: 100%|██████████| 5.85k/5.85k [00:00<00:00, 16.9MB/s]


model.safetensors: 100%|█████████▉| 54.3M/54.3M [00:00<00:00, 135MB/s]
Some weights of the model checkpoint at Raghavan/ic17mlt_Fast_T were not used when initializing FastForSceneTextRecognition: ['backbone.textnet.first_conv.batch_norm.bias', 'backbone.textnet.first_conv.batch_norm.num_batches_tracked', 'backbone.textnet.first_conv.batch_norm.running_mean', 'backbone.textnet.first_conv.batch_norm.running_var', 'backbone.textnet.first_conv.batch_norm.weight', 'backbone.textnet.first_conv.conv.weight', 'backbone.textnet.stage1.0.horizontal_batch_norm.bias', 'backbone.textnet.stage1.0.horizontal_batch_norm.num_batches_tracked', 'backbone.textnet.stage1.0.horizontal_batch_norm.running_mean', 'backbone.textnet.stage1.0.horizontal_batch_norm.running_var', 'backbone.textnet.stage1.0.horizontal_batch_norm.weight', 'backbone.textnet.stage1.0.horizontal_conv.weight', 'backbone.textnet.stage1.0.main_batch_norm.bias', 'backbone.textnet.stage1.0.main_batch_norm.num_batches_tracked', 'backbone.tex

In [3]:
import torch
inputs = processor(image, return_tensors="pt")
# forward pass
with torch.no_grad():
    outputs = model(pixel_values=inputs["pixel_values"])
target_sizes = [(image.shape[1], image.shape[2]) for image in inputs["pixel_values"]]
threshold = 0.85
text_locations = processor.post_process_text_detection(outputs, target_sizes, threshold, bbox_type="poly")

/home/user/app/transformers/src/transformers/models/fast/modeling_fast.py:323: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  return F.upsample(layer_out, size=(height, width), mode="bilinear")


In [1]:
from huggingface_hub import hf_hub_download
import json
config_filepath = hf_hub_download(repo_id="Raghavan/fast_model_config_files", filename="fast_model_configs.json")

/home/user/miniconda/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config_model_file_path = hf_hub_download(repo_id="jadechoghari/fast-configs", filename="fast_tiny_ic17mlt_640.json")
with open(config_model_file_path) as f:
        content_model = json.loads(f.read())
model_config = content_model["model"]
test_config = content_model.get("test_cfg", None)
data_config = content_model["data"]

fast_tiny_ic17mlt_640.json: 100%|██████████| 1.75k/1.75k [00:00<00:00, 6.70MB/s]
